In [2]:
import os
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import Search

In [3]:
cwd = "c:\\Users\\m1865\\Desktop\\DISC"
cwd_Images_Raw = cwd + "\\Sentinel-2 Images Raw"
cwd_Images_Processed = cwd + "\\Sentinel-2 Images Processed"

In [4]:
# Site Name
df = pd.read_csv(cwd + "//site.csv")
df.head()

,Number,Site,Serial Number (FLOX),Country,Location,Latitude,Longitude,Priority,Plant functional type (PFT),Reference network,Number of components,Reduce ROI size (600 x 600 m),Move ROI,Comments,PRISMA Availability (900m Buffer ROI Check),PRISMA Earliest Date,PRISMA Latest Date (Before 2024-06-26),Tomi,S2 Cloud
0,1,ATGE,NaN,NaN,NaN,52.466778,12.959778,NaN,NaN,HYPERNET,NaN,NaN,NaN,NaN,2,2022-05-06,2023-08-14,Y,NaN
1,2,ATLAS-Mohammed V,NaN,Morocco,ATLAS-Mohammed V,33.406152,-5.103319,Low,NaN,NaN,NaN,Yes,Yes,"mixed pixel, complex topography",2,2023-11-23,2023-11-23,NaN,NaN
2,3,AU-MIEMING,JB-042-GW,Austria,Austria,47.316700,10.970300,NaN,NaN,FLOX,NaN,No,No,topography?,0,NaN,NaN,Duplicate,NaN
3,4,BASP,NaN,NaN,NaN,39.049139,-2.075917,NaN,NaN,HYPERNET,NaN,NaN,NaN,NaN,27,2022-05-09,2024-06-18,Y,NaN
4,5,BE-BRASCHAAT,JB-021-SW,Belgium,Brasschaat,51.307600,4.519900,NaN,NaN,FLOX,NaN,No,No,Check the site with Sentinal 3,7,2020-09-12,2022-06-01,NaN,NaN


In [5]:
gdf = gpd.GeoDataFrame(
    df[['Site','Latitude','Longitude']], 
    geometry=gpd.points_from_xy(df.Longitude, df.Latitude), 
    crs = "EPSG:4326"
)
gdf.head()

,Site,Latitude,Longitude,geometry
0,ATGE,52.466778,12.959778,POINT (12.95978 52.46678)
1,ATLAS-Mohammed V,33.406152,-5.103319,POINT (-5.10332 33.40615)
2,AU-MIEMING,47.316700,10.970300,POINT (10.97030 47.31670)
3,BASP,39.049139,-2.075917,POINT (-2.07592 39.04914)
4,BE-BRASCHAAT,51.307600,4.519900,POINT (4.51990 51.30760)


In [6]:
for i in range(df.shape[0]):
    temp_Name = df.loc[i,"Site"]
    temp_gdf = gpd.read_file(cwd_Images_Processed + "\\" + temp_Name + "\\100m.shp").to_crs(epsg = 4326)
    if i == 0:
        gdf_100 = temp_gdf.copy()
    else:
        gdf_100 = pd.concat([gdf_100,temp_gdf])
gdf_100.drop(columns = ['0'], inplace = True)
gdf_100['Site'] = list(gdf['Site'])
gdf_100 = gdf_100[['Site','geometry']]
gdf_100.head()

,Site,geometry
0,ATGE,"POLYGON ((12.96048 52.46629, 12.95887 52.46634..."
0,ATLAS-Mohammed V,"POLYGON ((-5.10271 33.40570, -5.10389 33.40568..."
0,AU-MIEMING,"POLYGON ((10.97103 47.31623, 10.96958 47.31625..."
0,BASP,"POLYGON ((-2.07527 39.04863, -2.07654 39.04864..."
0,BE-BRASCHAAT,"POLYGON ((4.52072 51.30706, 4.51915 51.30708, ..."


In [7]:
for i in range(df.shape[0]):
    temp_Name = df.loc[i,"Site"]
    temp_gdf = gpd.read_file(cwd_Images_Processed + "\\" + temp_Name + "\\300m.shp").to_crs(epsg = 4326)
    if i == 0:
        gdf_300 = temp_gdf.copy()
    else:
        gdf_300 = pd.concat([gdf_300,temp_gdf])
gdf_300.drop(columns = ['0'], inplace = True)
gdf_300['Site'] = list(gdf['Site'])
gdf_300 = gdf_300[['Site','geometry']]
gdf_300.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [8]:
a = gpd.read_file(cwd_Images_Processed + "\\ATLAS-Mohammed V\\600m Spostamento.shp")
a.crs

<Projected CRS: EPSG:32630>
Name: WGS 84 / UTM zone 30N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 6°W and 0°W, northern hemisphere between equator and 84°N, onshore and offshore. Algeria. Burkina Faso. Côte' Ivoire (Ivory Coast). Faroe Islands - offshore. France. Ghana. Gibraltar. Ireland - offshore Irish Sea. Mali. Mauritania. Morocco. Spain. United Kingdom (UK).
- bounds: (-6.0, 0.0, 0.0, 84.0)
Coordinate Operation:
- name: UTM zone 30N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [9]:
list_SiteMoved = []
for i in range(df.shape[0]):
    temp_Name = df.loc[i,"Site"]
    if os.path.exists(cwd_Images_Processed + "\\" + temp_Name + "\\600m Spostamento.shp"):
        list_SiteMoved.append(temp_Name)
        temp_gdf = gpd.read_file(cwd_Images_Processed + "\\" + temp_Name + "\\600m Spostamento.shp")
        temp_gdf = temp_gdf.to_crs(epsg = 4326)
        if len(list_SiteMoved) == 1:
            print("Initialiazation!")
            gdf_600 = temp_gdf.copy()
        else:
            gdf_600 = pd.concat([gdf_600,temp_gdf])
    elif os.path.exists(cwd_Images_Processed + "\\" + temp_Name + "\\600m.shp"): 
        os.path.exists(cwd_Images_Processed + "\\" + temp_Name + "\\600m.shp")
        list_SiteMoved.append(temp_Name)
        temp_gdf = gpd.read_file(cwd_Images_Processed + "\\" + temp_Name + "\\600m.shp")
        temp_gdf = temp_gdf.to_crs(epsg = 4326)
        if len(list_SiteMoved) == 1:
            print("Initialiazation!")
            gdf_600 = temp_gdf.copy()
        else:
            gdf_600 = pd.concat([gdf_600,temp_gdf])

gdf_600.drop(columns = ['0'], inplace = True)
gdf_600['Site'] = list_SiteMoved
gdf_600 = gdf_600[['Site','geometry']]
gdf_600

Initialiazation!


,Site,geometry
0,ATLAS-Mohammed V,"POLYGON ((-5.09833 33.40216, -5.10477 33.40205..."
0,BE-Vie,"POLYGON ((6.00220 50.30208, 5.99365 50.30230, ..."
0,DE-GEBESE,"POLYGON ((10.91672 51.09692, 10.90802 51.09706..."
0,DE-SEL,"POLYGON ((6.45165 50.86323, 6.44299 50.86304, ..."
0,FR-Bil,"POLYGON ((-0.95423 44.49087, -0.96190 44.49100..."
0,FR-FBn,"POLYGON ((5.68403 43.23788, 5.67652 43.23806, ..."
0,FR-Pue,"POLYGON ((3.60119 43.73981, 3.59361 43.73985, ..."
0,IS-YATIR,"POLYGON ((35.05285 31.34376, 35.04644 31.34386..."
0,IT-BFt,"POLYGON ((10.74331 45.19589, 10.73554 45.19601..."
0,IT-SR2,"POLYGON ((10.29464 43.72793, 10.28707 43.72802..."


In [10]:
for i in range(df.shape[0]):
    temp_Name = df.loc[i,"Site"]
    temp_gdf = gpd.read_file(cwd_Images_Processed + "\\" + temp_Name + "\\900m.shp").to_crs(epsg = 4326)
    if i == 0:
        gdf_900 = temp_gdf.copy()
    else:
        gdf_900 = pd.concat([gdf_900,temp_gdf])
gdf_900.drop(columns = ['0'], inplace = True)
gdf_900['Site'] = list(gdf['Site'])
gdf_900 = gdf_900[['Site','geometry']]
gdf_900.head()

,Site,geometry
0,ATGE,"POLYGON ((12.96603 52.46250, 12.95267 52.46295..."
0,ATLAS-Mohammed V,"POLYGON ((-5.09833 33.40216, -5.10810 33.40200..."
0,AU-MIEMING,"POLYGON ((10.97619 47.31254, 10.96415 47.31274..."
0,BASP,"POLYGON ((-2.07069 39.04499, -2.08121 39.04507..."
0,BE-BRASCHAAT,"POLYGON ((4.52634 51.30339, 4.51329 51.30356, ..."


In [11]:
for i in range(df.shape[0]):
    temp_Name = df.loc[i,"Site"]
    temp_gdf = gpd.read_file(cwd_Images_Processed + "\\" + temp_Name + "\\1200m.shp").to_crs(epsg = 4326)
    if i == 0:
        gdf_1200 = temp_gdf.copy()
    else:
        gdf_1200 = pd.concat([gdf_1200,temp_gdf])
gdf_1200.drop(columns = ['0'], inplace = True)
gdf_1200['Site'] = list(gdf['Site'])
gdf_1200 = gdf_1200[['Site','geometry']]
gdf_1200.head()

,Site,geometry
0,ATGE,"POLYGON ((12.96811 52.46109, 12.95035 52.46168..."
0,ATLAS-Mohammed V,"POLYGON ((-5.09668 33.40084, -5.10968 33.40062..."
0,AU-MIEMING,"POLYGON ((10.97812 47.31115, 10.96212 47.31143..."
0,BASP,"POLYGON ((-2.06898 39.04362, -2.08296 39.04374..."
0,BE-BRASCHAAT,"POLYGON ((4.52845 51.30201, 4.51110 51.30224, ..."


In [12]:
m = folium.Map(location=[47.316500, 10.970300], zoom_start=4, tiles="Esri.WorldImagery")

# Build folium geojson (site point)
geo_site = folium.GeoJson(
    gdf,
    name="Sites",
    # style_function=style_function,
    tooltip=folium.GeoJsonTooltip(
        fields=["Site", "Latitude", "Longitude"], localize=True
    ),
).add_to(m)

# Build folium geojson (1200m polygon)
geo_1200m = folium.GeoJson(
    gdf_1200,
    name="1200m",
    style_function= lambda x: {'fillColor': '#00000000', 'color': 'Red'},
    popup = folium.Popup("1200m x 1200m!")
).add_to(m)

# Build folium geojson (900m polygon)
geo_900m = folium.GeoJson(
    gdf_900,
    name="900m",
    style_function= lambda x: {'fillColor': '#00000000', 'color': 'Yellow'},
    popup = folium.Popup("900m x 900m!")
).add_to(m)

# Build folium geojson (300m polygon)
geo_600m = folium.GeoJson(
    gdf_600,
    name="600m Moved",
    style_function= lambda x: {'fillColor': '#00000000', 'color': 'Green'},
    popup = folium.Popup("600m x 600m!")
).add_to(m)


# Build folium geojson (300m polygon)
geo_300m = folium.GeoJson(
    gdf_300,
    name="300m",
    style_function= lambda x: {'fillColor': '#00000000', 'color': 'Orange'},
    popup = folium.Popup("300m x 300m!")
).add_to(m)


# Build folium geojson (100m polygon)
geo_100m = folium.GeoJson(
    gdf_100,
    name="100m",
    style_function= lambda x: {'fillColor': '#00000000', 'color': 'Cyan'},
    popup = folium.Popup("100m x 100m!")
).add_to(m)


# Search
statesearch = Search(
    layer=geo_site,
    geom_type="Point",
    placeholder="Search for a site",
    collapsed=False,
    search_label="Site",
    search_zoom = 15,
    weight=3,
).add_to(m)

folium.LayerControl().add_to(m)
m

In [13]:
m.save(cwd + "\\Site Map.html")

In [14]:
# Quick export!!!
import io
from PIL import Image
for i in range(df.shape[0]):
    site = df.loc[i,'Site']
    lat = df.loc[i,'Latitude']
    lon = df.loc[i, 'Longitude']
    m = folium.Map(location=[lat, lon], zoom_start=15, tiles="Esri.WorldImagery")

    # Build folium geojson (site point)
    geo_site = folium.GeoJson(
        gdf,
        name="Sites",
        # style_function=style_function,
        tooltip=folium.GeoJsonTooltip(
            fields=["Site", "Latitude", "Longitude"], localize=True
        ),
    ).add_to(m)

    # Build folium geojson (1200m polygon)
    geo_1200m = folium.GeoJson(
        gdf_1200,
        name="1200m",
        style_function= lambda x: {'fillColor': '#00000000', 'color': 'Red'},
        popup = folium.Popup("1200m x 1200m!")
    ).add_to(m)

    # Build folium geojson (900m polygon)
    geo_900m = folium.GeoJson(
        gdf_900,
        name="900m",
        style_function= lambda x: {'fillColor': '#00000000', 'color': 'Yellow'},
        popup = folium.Popup("900m x 900m!")
    ).add_to(m)

    # Build folium geojson (300m polygon)
    geo_600m = folium.GeoJson(
        gdf_600,
        name="600m Moved",
        style_function= lambda x: {'fillColor': '#00000000', 'color': 'Green'},
        popup = folium.Popup("600m x 600m!")
    ).add_to(m)


    # Build folium geojson (300m polygon)
    geo_300m = folium.GeoJson(
        gdf_300,
        name="300m",
        style_function= lambda x: {'fillColor': '#00000000', 'color': 'Orange'},
        popup = folium.Popup("300m x 300m!")
    ).add_to(m)


    # Build folium geojson (100m polygon)
    geo_100m = folium.GeoJson(
        gdf_100,
        name="100m",
        style_function= lambda x: {'fillColor': '#00000000', 'color': 'Cyan'},
        popup = folium.Popup("100m x 100m!")
    ).add_to(m)

    img_data = m._to_png(5)
    img = Image.open(io.BytesIO(img_data))
    img.save(cwd_Images_Processed + "\\" + site + "\\map.png")
    print(f"{site} Generated!")

ATGE Generated!
ATLAS-Mohammed V Generated!
AU-MIEMING Generated!
BASP Generated!
BE-BRASCHAAT Generated!
BE-Maa Generated!
BE-Vie Generated!
BE-WAL Generated!
CANADA-ONT Generated!
CD-Ygb Generated!
CHINA-AGR 1 Generated!
CHINA-AGR 2 Generated!
CHINA-FOR 1 Generated!
CZ-Lnz Generated!
Dahara (Senegal) Generated!
DEGE Generated!
DE-GEBESE Generated!
DE-Hai Generated!
DE-JUL1 Generated!
DE-KKA Generated!
DE-LEINIEFELDE Generated!
DE-LEIPZIG Generated!
DE-LEIPZIG 2 Generated!
DE-SEL Generated!
DE-Tha Generated!
DK-Sor Generated!
FI Hyt FOREST Generated!
FI SOD FOREST Generated!
FI SOD GRASS Generated!
FI-Ken Generated!
FINLAND SWAMP Generated!
FI-Var Generated!
FR-Bil Generated!
FR-FBn Generated!
FR-Hes Generated!
FR-OHP Generated!
FR-Pue Generated!
GF-Guy Generated!
GHNA Generated!
IFAR Generated!
INDONESIA Generated!
IN-JOD Generated!
IS-YATIR Generated!
IT-BFt Generated!
IT-BOZEN Generated!
IT-Cp2 Generated!
IT-Mbo Generated!
IT-SR2 Generated!
IT-TOR Generated!
JAES Generated!
KEN-1 G